In [1]:
# Download YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies
%cd ../

Cloning into 'yolov5'...
remote: Enumerating objects: 9839, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9839 (delta 3), reused 5 (delta 3), pack-reused 9828
Receiving objects: 100% (9839/9839), 10.03 MiB | 27.69 MiB/s, done.
Resolving deltas: 100% (6847/6847), done.
/kaggle/working/yolov5
Note: you may need to restart the kernel to use updated packages.
/kaggle/working


In [2]:
import os
import gc
import cv2
import yaml
import torch
import random
import shutil
import warnings
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image, ImageDraw
from shutil import copyfile
from IPython.core.display import Video, display
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold

warnings.simplefilter('ignore')
pd.set_option("max_columns", 150)
pd.set_option('display.max_rows', 150)
# turn off W&B syncing if you don't need
os.environ['WANDB_MODE'] = 'offline'

### Configuration

In [3]:
CFG = {
    "seed"       : 42,
    "model"      : "yolov5s",
    "input_path" : '../input/nfl-health-and-safety-helmet-assignment/images/',
    "output_path": './nfl/',
    "batch_size" : 24,
    "epochs"     : 5,
    "size"       : 800,
    "fold_num"   : 3
}

CFG

{'seed': 42,
 'model': 'yolov5s',
 'input_path': '../input/nfl-health-and-safety-helmet-assignment/images/',
 'output_path': './nfl/',
 'batch_size': 24,
 'epochs': 5,
 'size': 800,
 'fold_num': 3}

In [4]:
def seed_everything(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(CFG["seed"])

### Load data

In [5]:
# Load image level csv file
df_train = pd.read_csv('../input/nfl-health-and-safety-helmet-assignment/image_labels.csv')
df_train = df_train[df_train.label != "Helmet-Sideline"].reset_index(drop=True)
print(df_train.label.value_counts())

df_train["right"]       = df_train.left + df_train.width
df_train["bottom"]      = df_train.top  + df_train.height
df_train["video_frame"] = df_train.image.apply(lambda x: "_".join(x.split("_")[:3]))

print(df_train.shape)
df_train.head()

Helmet              129764
Helmet-Blurred       33544
Helmet-Partial        8814
Helmet-Difficult      6577
Name: label, dtype: int64
(178699, 9)


,image,label,left,width,top,height,right,bottom,video_frame
0,57503_000116_Endzone_frame443.jpg,Helmet,1099,16,456,15,1115,471,57503_000116_Endzone
1,57503_000116_Endzone_frame443.jpg,Helmet,1117,15,478,16,1132,494,57503_000116_Endzone
2,57503_000116_Endzone_frame443.jpg,Helmet,828,16,511,15,844,526,57503_000116_Endzone
3,57503_000116_Endzone_frame443.jpg,Helmet,746,16,519,16,762,535,57503_000116_Endzone
4,57503_000116_Endzone_frame443.jpg,Helmet,678,17,554,17,695,571,57503_000116_Endzone


In [6]:
df_min_area    = df_train.groupby(["image","video_frame"], as_index=False)[["left","top"]].min()
df_max_area    = df_train.groupby(["image","video_frame"], as_index=False)[["right","bottom","width","height"]].max()
df_area_detect = df_min_area.merge(df_max_area, on=["image","video_frame"])
df_area_detect.left   = df_area_detect.apply(lambda x: x.left   - x.width  if x.width  < x.left else 0, axis=1)
df_area_detect.right  = df_area_detect.apply(lambda x: x.right  + x.width  if x.right  + x.width < 1280 else 1280, axis=1)
df_area_detect.top    = df_area_detect.apply(lambda x: x.top    - x.height if x.height < x.top  else 0, axis=1)
df_area_detect.bottom = df_area_detect.apply(lambda x: x.bottom + x.height if x.bottom + x.height < 720 else 720,  axis=1)

df_area_detect.width  = df_area_detect.right  - df_area_detect.left
df_area_detect.height = df_area_detect.bottom - df_area_detect.top
df_area_detect["x_center"] = df_area_detect.left + (df_area_detect.width  / 2).astype(int)
df_area_detect["y_center"] = df_area_detect.top  + (df_area_detect.height / 2).astype(int)
df_area_detect.width    /= 1280
df_area_detect.height   /= 720
df_area_detect.x_center /= 1280
df_area_detect.y_center /= 720

df_area_detect["label"]    = "DetectArea"
df_area_detect["label_id"] = 0
df_train = df_area_detect.copy()

print(df_train.shape)
df_train.head(2)

(9930, 12)


,image,video_frame,left,top,right,bottom,width,height,x_center,y_center,label,label_id
0,57502_000480_Endzone_frame0495.jpg,57502_000480_Endzone,153,89,1135,521,0.767188,0.600000,0.503125,0.423611,DetectArea,0
1,57502_001570_Sideline_frame1395.jpg,57502_001570_Sideline,62,137,979,532,0.716406,0.548611,0.406250,0.463889,DetectArea,0


In [7]:
kf = GroupKFold(n_splits=CFG["fold_num"])
for i, (tr_idx, va_idx) in enumerate(kf.split(df_train, None, df_train.video_frame)):
    df_train[f"fold{i}"] = 0
    df_train.loc[tr_idx, f"fold{i}"] = 1
    _sum = df_train[f"fold{i}"].sum()
    print(f"Fold {i} train: {_sum}, val: {df_train.shape[0] - _sum}")
    
df_train.head()

Fold 0 train: 6620, val: 3310
Fold 1 train: 6620, val: 3310
Fold 2 train: 6620, val: 3310


,image,video_frame,left,top,right,bottom,width,height,x_center,y_center,label,label_id,fold0,fold1,fold2
0,57502_000480_Endzone_frame0495.jpg,57502_000480_Endzone,153,89,1135,521,0.767188,0.600000,0.503125,0.423611,DetectArea,0,1,1,0
1,57502_001570_Sideline_frame1395.jpg,57502_001570_Sideline,62,137,979,532,0.716406,0.548611,0.406250,0.463889,DetectArea,0,0,1,1
2,57502_002557_Sideline_frame0746.jpg,57502_002557_Sideline,112,140,1152,720,0.812500,0.805556,0.493750,0.597222,DetectArea,0,1,1,0
3,57502_002958_Endzone_frame0584.jpg,57502_002958_Endzone,311,208,1280,444,0.757031,0.327778,0.621094,0.452778,DetectArea,0,1,0,1
4,57502_003762_Endzone_frame1071.jpg,57502_003762_Endzone,75,0,1275,493,0.937500,0.684722,0.527344,0.341667,DetectArea,0,0,1,1


### Make YOLO configuration files

In [8]:
def create_file(df, fold, input_path, output_path):
    
    def make(_df, tr_val):
        for img_name in tqdm(_df.image.unique()):
            df_this_frame = _df[_df['image']==img_name]
            shutil.copy(f'{input_path}{img_name}',
                        f'{output_path}images/{tr_val}/{fold}/{img_name}')
            # Crop the side that there are no helmets
            with open(f'{output_path}labels/{tr_val}/{fold}/{img_name[:-4]}.txt', 'w+') as f:
                row = df_this_frame[['label_id','x_center','y_center','width','height']]
                row = row.values.astype('str')
                for box in range(len(row)):
                    text = ' '.join(row[box])
                    f.write(text)
                    f.write('\n')
        
    os.makedirs(f'{output_path}labels/train/{fold}/', exist_ok=True)
    os.makedirs(f'{output_path}images/train/{fold}/', exist_ok=True)
    os.makedirs(f'{output_path}labels/val/{fold}/',   exist_ok=True)
    os.makedirs(f'{output_path}images/val/{fold}/',   exist_ok=True)
    
    train_df = df[df[f'fold{fold}']==1].reset_index(drop=True)
    val_df   = df[df[f'fold{fold}']==0].reset_index(drop=True)
    make(train_df, "train")
    make(val_df,   "val")

In [9]:
class_names = list(df_train.label.unique())

def make_training_yaml(fold):
    with open(f"./yolov5_{fold}.yaml", "w+") as yf:
        yaml.dump({
            "train": f"{CFG['output_path']}images/train/{fold}/",
            "val"  : f"{CFG['output_path']}images/val/{fold}/",
            "nc"   : 1,
            "names": class_names
        }, yf, default_flow_style=False)

In [10]:
!cp ./yolov5/models/{CFG['model']}.yaml ./{CFG['model']}.yaml

with open(f"./{CFG['model']}.yaml", "r+") as yf:
    opened_yf = yaml.load(yf)
    !rm -rf ./{CFG['model']}.yaml
    opened_yf["nc"] = 1
    with open(f"./{CFG['model']}.yaml", "w") as yf2:
        yaml.dump(opened_yf, yf2, default_flow_style=False)

In [11]:
!cat ./yolov5/data/hyps/hyp.scratch.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (effic

### Run YOLO training

In [12]:
!rm -rf ./nfl
!rm -rf ./yolov5/runs
!rm -rf ./runs*

for i in range(CFG["fold_num"]):
    print("Making a yaml file for training and validation data...")
    create_file(df_train, i, CFG['input_path'], CFG['output_path'])
    print("Making a yaml file for configuration of YOLOv5...")
    make_training_yaml(i)

    print("Running YOLOv5 training...")
    !python ./yolov5/train.py \
        --img     {CFG["size"]} \
        --weights {CFG["model"]}.pt \
        --batch   {CFG["batch_size"]} \
        --epochs  {CFG["epochs"]} \
        --hyp     ./yolov5/data/hyps/hyp.scratch.yaml \
        --data    ./yolov5_{i}.yaml \
        --cfg     ./{CFG["model"]}.yaml
    
    !rm -rf ./nfl/images/train/{i}
    !rm -rf ./wandb/
    !mv ./yolov5/runs ./runs_{i}
    print("Done")

  0%|          | 0/6620 [00:00<?, ?it/s]

Making a yaml file for training and validation data...


100%|██████████| 3310/3310 [00:31<00:00, 106.53it/s]


Making a yaml file for configuration of YOLOv5...
Running YOLOv5 training...
train: weights=yolov5s.pt, cfg=./yolov5s.yaml, data=./yolov5_0.yaml, hyp=./yolov5/data/hyps/hyp.scratch.yaml, epochs=5, batch_size=24, imgsz=800, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
2021-10-18 16:14:52.104143: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-18 16:14:56.792611: I tensorflow/stream_executor/platform/default/dso_loader.

  0%|          | 0/6620 [00:00<?, ?it/s]

...

Making a yaml file for configuration of YOLOv5...
Running YOLOv5 training...
train: weights=yolov5s.pt, cfg=./yolov5s.yaml, data=./yolov5_1.yaml, hyp=./yolov5/data/hyps/hyp.scratch.yaml, epochs=5, batch_size=24, imgsz=800, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
2021-10-18 17:22:02.751399: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-18 17:22:05.358051: I tensorflow/stream_executor/platform/default/dso_loader.

  0%|          | 0/6620 [00:00<?, ?it/s]

Done
Making a yaml file for training and validation data...


100%|██████████| 3310/3310 [00:19<00:00, 170.97it/s]


Making a yaml file for configuration of YOLOv5...
Running YOLOv5 training...
train: weights=yolov5s.pt, cfg=./yolov5s.yaml, data=./yolov5_2.yaml, hyp=./yolov5/data/hyps/hyp.scratch.yaml, epochs=5, batch_size=24, imgsz=800, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
2021-10-18 18:29:10.467738: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-18 18:29:13.041127: I tensorflow/stream_executor/platform/default/dso_loader.

In [13]:
for i in range(CFG["fold_num"]):
    !mkdir result_{i}
    !cp ./runs_{i}/train/exp/results*  ./result_{i}/
    !cp ./runs_{i}/train/exp/weights/* ./result_{i}/

In [14]:
!rm -rf yolov5_* {CFG["model"]}*
!rm -rf ./yolov5
!rm -rf ./nfl/
!rm -rf ./runs_*